<a href="https://colab.research.google.com/github/suryagokul/Deep-Learning/blob/main/cnn_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Convolutional Neural Network

`Here we are saying to use only 50% of gpu memory.After 50 percent of memory is used,then deallocate some of the memory and reallocate again.`

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
# Convolutional Neural Network

# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.4.0'

In [6]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Part 1 - Data Preprocessing

# Preprocessing the Training set or Data Augmentation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


In [7]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 456 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [8]:
from tensorflow.keras.layers import Conv2D

In [10]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [11]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [12]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
cnn.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
15/15 [==============================] - 255s 17s/step - loss: 0.8784 - accuracy: 0.5442 - val_loss: 0.6903 - val_accuracy: 0.5250
Epoch 2/10
15/15 [==============================] - 4s 290ms/step - loss: 0.6664 - accuracy: 0.6216 - val_loss: 0.6843 - val_accuracy: 0.5250
Epoch 3/10
15/15 [==============================] - 4s 289ms/step - loss: 0.6640 - accuracy: 0.6181 - val_loss: 0.6912 - val_accuracy: 0.5250
Epoch 4/10
15/15 [==============================] - 4s 311ms/step - loss: 0.6382 - accuracy: 0.6520 - val_loss: 0.6642 - val_accuracy: 0.6500
Epoch 5/10
15/15 [==============================] - 5s 294ms/step - loss: 0.6571 - accuracy: 0.6512 - val_loss: 0.6379 - val_accuracy: 0.6000
Epoch 6/10
15/15 [==============================] - 4s 288ms/step - loss: 0.6277 - accuracy: 0.6714 - val_loss: 0.6022 - val_accuracy: 0.7000
Epoch 7/10
15/15 [==============================] - 4s 288ms/step - loss: 0.6242 - accuracy: 0.6740 - val_loss: 0.5954 - val_accuracy: 0.7250
Epoch 

In [13]:
# save it as a h5 file

cnn.save('model_rcat_dog.h5')

In [14]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model('model_rcat_dog.h5')

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [16]:
# Part 4 - Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/Datasets/test/cats/images (25).jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

In [20]:
test_image.shape

(1, 64, 64, 3)

In [21]:
result

array([[0.26227996]], dtype=float32)

In [22]:
if result[0]<=0.5:
    print("The image classified is cat")
else:
    print("The image classified is dog")

The image classified is cat
